In [1]:
import pandas as pd
data = pd.read_csv("csxl_data.csv")

In [2]:
import altair as alt

In [6]:
data['start'] = pd.to_datetime(data['start'])

# Get the day of the week
data['day_of_week'] = data['start'].dt.day_name()

# Get the hour of the day in non-military time (also remove leading 0s, for example, 01pm -> 1pm)
data['civilian_time'] = data['start'].dt.strftime('%I %p').str.lstrip("0")

# Group reservations by the day of the week + hour of the day + count the number of different reservations
reservations_per_hour = data.groupby(['day_of_week', 'civilian_time']).size().reset_index(name='count')

chart = alt.Chart(reservations_per_hour).mark_bar().encode(
    x=alt.X('civilian_time:N', title='Hour of the Day', sort=['10 AM', '11 AM', '12 PM', '1 PM', '2 PM', '3 PM', '4 PM', '5 PM']),
    y=alt.Y('count:Q', title=None),
    # NOTE: Saturday and Sunday are not currently included (no reservations on weekends?)
    column=alt.Column('day_of_week:N', title=None, sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
).properties(
    height=150,
    title='Popular Times in the CSXL'
).configure_axisY(
    labels=False,
    grid=False
)

chart.show()

alt.Chart(...)